# EV Calculators


In [1]:
import pandas as pd
import numpy as np

# Load the Excel file
data = pd.read_excel("C://Users//yasir//OneDrive//Desktop//research paper//project_ev1.xlsx", sheet_name='Sheet1')

# Extract relevant columns
relevant_columns = ['Car Name', 'Battery Capacity (kWh)', 'Range (km)']
data = data[relevant_columns]

In [2]:
# Electricity rates in India
data1 = {
    'state': [
        'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Delhi',
        'Goa', 'Gujarat', 'Haryana', 'Himanchal Pradesh', 'Jharkhand', 'Karnataka',
        'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram',
        'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana',
        'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal'
    ],
    'cost': [
        7.1, 5.6, 7.6, 8.1, 5.9, 6, 6.3, 5.45, 7.1, 5.65, 6.25, 8.2, 6.4, 6.74, 13.21,
        6.75, 6.1, 8.2, 6.4, 6.2, 6.7, 7.35, 5.6, 7.5, 7.75, 7.3, 6.2, 5.6, 7.3
    ]
}

# Create DataFrame
state_electricity_cost = pd.DataFrame(data1)


In [3]:
# Petrol prices data
petrol_data = {
    "State/UT": [
        "Andaman & Nicobar Islands", "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh",
        "Delhi", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jammu & Kashmir", "Jharkhand", "Karnataka",
        "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", 
        "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Uttar Pradesh", "Uttarakhand", "West Bengal"
    ],
    "Petrol Price (₹/Liter)": [
        82.42, 109.53, 93.13, 97.90, 106.27, 101.61, 94.72, 96.43, 94.83, 95.29, 94.51, 98.12, 98.60, 
        103.44, 106.35, 107.46, 104.70, 99.72, 95.96, 94.92, 98.13, 102.59, 96.58, 105.56, 101.50, 101.79, 
        95.03, 94.05, 105.38
    ]
}

# Diesel prices data
diesel_data = {
    "State/UT": [
        "Andaman & Nicobar Islands", "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh",
        "Delhi", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jammu & Kashmir", "Jharkhand", "Karnataka",
        "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", 
        "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Uttar Pradesh", "Uttarakhand", "West Bengal"
    ],
    "Diesel Price (₹/Liter)": [
        78.01, 97.35, 82.43, 90.13, 93.06, 94.54, 87.62, 88.20, 90.51, 88.13, 86.75, 83.59, 93.34, 
        89.44, 95.30, 92.77, 91.22, 85.73, 86.80, 81.37, 89.36, 94.11, 86.83, 90.96, 88.80, 93.38, 
        88.16, 88.95, 92.16
    ]
}

# Creating the petrol dataframe
petrol_df = pd.DataFrame(petrol_data)

# Creating the diesel dataframe
diesel_df = pd.DataFrame(diesel_data)

# Adding new data for Telangana and Tripura to petrol data
additional_petrol_data = {
    "State/UT": ["Telangana", "Tripura"],
    "Petrol Price (₹/Liter)": [108.20, 96.65]
}

# Adding new data for Telangana and Tripura to diesel data
additional_diesel_data = {
    "State/UT": ["Telangana", "Tripura"],
    "Diesel Price (₹/Liter)": [94.62, 87.45]
}

# Append the additional data to the existing dataframes
petrol_df = pd.concat([petrol_df, pd.DataFrame(additional_petrol_data)], ignore_index=True)
diesel_df = pd.concat([diesel_df, pd.DataFrame(additional_diesel_data)], ignore_index=True)


# Model 1 - Battery Range Calculator

In [8]:

def get_user_inputs():
    # Get user inputs
    try:
        car_name = input("Enter the car name: ").strip()
        speed_kmph = float(input("Enter the speed in kilometers per hour (in multiples of 10): "))
        temperature_celsius = float(input("Enter the temperature in Celsius (ranging from 0°C - 50°C in multiples of 5): "))
        ac_status = input("Is the A/C on? (yes/no): ").strip().lower() == 'yes'
    except ValueError:
        print("Invalid input. Please enter numeric values for speed and temperature.")
        return None

    return car_name, speed_kmph, temperature_celsius, ac_status

def get_car_range(car_name):
    # Get the range for the car based on its name
    car_info = data[data['Car Name'].str.lower() == car_name.lower()]
    if not car_info.empty:
        return car_info['Range (km)'].values[0]
    else:
        print("Car not found. Please enter a valid car name.")
        return None

def display_inputs(car_name, range_km, speed_kmph, temperature_celsius, ac_status):
    # Display the inputs
    print(f"Car Name: {car_name}")
    print(f" Expected Range: {range_km} km")
    print(f"Speed: {speed_kmph} km/h")
    print(f"Temperature: {temperature_celsius} °C")
    print(f"A/C Status: {'On' if ac_status else 'Off'}")

def calculate_actual_range(range_km, speed_kmph, temperature_celsius, ac_status):
    # Calculate speed penalty
    speed_penalty = 0
    if speed_kmph in [50, 90]:
        speed_penalty = 0.005 * range_km
    elif speed_kmph in [40, 100]:
        speed_penalty = 0.01 * range_km
    elif speed_kmph in [30, 110]:
        speed_penalty = 0.015 * range_km
    elif speed_kmph in [20, 120]:
        speed_penalty = 0.02 * range_km

    # Calculate temperature penalty
    temperature_penalty = 0
    if temperature_celsius in [15, 30]:
        temperature_penalty = 0.01 * range_km
    elif temperature_celsius in [10, 35]:
        temperature_penalty = 0.02 * range_km
    elif temperature_celsius in [5, 40]:
        temperature_penalty = 0.03 * range_km
    elif temperature_celsius in [0, 45]:
        temperature_penalty = 0.03 * range_km
    elif temperature_celsius == 50:
        temperature_penalty = 0.04 * range_km

    # Calculate A/C penalty
    ac_penalty = 0.1 * range_km if ac_status else 0

    # Calculate the actual range
    actual_range = range_km - speed_penalty - temperature_penalty - ac_penalty
    return actual_range

def main():
    user_inputs = get_user_inputs()
    if user_inputs:
        car_name, speed_kmph, temperature_celsius, ac_status = user_inputs
        range_km = get_car_range(car_name)
        
        if range_km is not None:
            display_inputs(car_name, range_km, speed_kmph, temperature_celsius, ac_status)
            actual_range = calculate_actual_range(range_km, speed_kmph, temperature_celsius, ac_status)
            print(f"Actual Range: {actual_range:.2f} km")

if __name__ == "__main__":
    main()


Enter the car name: Tata Nexon EV Empowered Plus LR
Enter the speed in kilometers per hour (in multiples of 10): 70
Enter the temperature in Celsius (ranging from 0°C - 50°C in multiples of 5): 45
Is the A/C on? (yes/no): yes
Car Name: Tata Nexon EV Empowered Plus LR
 Expected Range: 465.0 km
Speed: 70.0 km/h
Temperature: 45.0 °C
A/C Status: On
Actual Range: 404.55 km


# Model 2 - Charging Cost Calculator

In [12]:
# Create a new column 'efficiency'
data['efficiency'] = data['Range (km)'] / data['Battery Capacity (kWh)']

# Function to get cost for a state
def get_electricity_cost(state):
    cost = state_electricity_cost.loc[state_electricity_cost['state'] == state, 'cost']
    if not cost.empty:
        return cost.values[0]
    else:
        raise ValueError(f"State '{state}' not found in the table")

# Function to get efficiency based on battery capacity
def get_efficiency(battery_capacity):
    efficiency = data.loc[data['Battery Capacity (kWh)'] == battery_capacity, 'efficiency']
    if not efficiency.empty:
        return efficiency.values[0]
    else:
        raise ValueError(f"Efficiency for battery capacity '{battery_capacity}' kWh not found")

# Function to get battery capacity from car name
def get_battery_capacity(car_name):
    battery_capacity = data.loc[data['Car Name'] == car_name, 'Battery Capacity (kWh)']
    if not battery_capacity.empty:
        return battery_capacity.values[0]
    else:
        raise ValueError(f"Car '{car_name}' not found in the data")

# Function to calculate costs
def calculate_costs(state, battery_capacity):
    electricity_cost = get_electricity_cost(state)
    efficiency = get_efficiency(battery_capacity)
    
    # Calculate the cost per full charge
    cost_per_charge = electricity_cost * battery_capacity
    
    # Calculate the cost per Km
    cost_per_km = cost_per_charge / (battery_capacity * efficiency)
    
    result = {
        'cost_per_charge': cost_per_charge,
        'cost_per_km': cost_per_km
    }
    
    return result

# User input for car name
car_name = input("Enter the car name: ")

# Retrieve the corresponding battery capacity
try:
    battery_capacity = get_battery_capacity(car_name)
    print(f"Battery Capacity for {car_name}: {battery_capacity} kWh")
    
    # User input for state
    state = input("Enter the state name: ")

    # Calculate and display costs
    try:
        costs = calculate_costs(state, battery_capacity)
        print(f"Cost per full charge: {costs['cost_per_charge']} Rupees")
        print(f"Cost per Km: {costs['cost_per_km']} Rupees")
    
    except ValueError as e:
        print(e)

except ValueError as e:
    print(e)


Enter the car name: BYD Seal Performance
Battery Capacity for BYD Seal Performance: 82.56 kWh
Enter the state name: Karnataka
Cost per full charge: 676.992 Rupees
Cost per Km: 1.1672275862068966 Rupees


# Model 3 - Saving's Calculator

In [13]:
def savings_calculator(car_name, fuel_type, state, kms_month, mileage):
    # Step 1: Retrieve EV details
    car_info = data[data['Car Name'] == car_name]
    if car_info.empty:
        return "Car name not found in the database."
    
    battery_capacity = car_info.iloc[0]['Battery Capacity (kWh)']
    range_km = car_info.iloc[0]['Range (km)']
    
    # Step 2: Retrieve Fuel Price
    if fuel_type.lower() == 'petrol':
        fuel_info = petrol_df[petrol_df['State/UT'] == state]
        if fuel_info.empty:
            return "State not found in petrol data."
        fuel_price = fuel_info.iloc[0]['Petrol Price (₹/Liter)']
    elif fuel_type.lower() == 'diesel':
        fuel_info = diesel_df[diesel_df['State/UT'] == state]
        if fuel_info.empty:
            return "State not found in diesel data."
        fuel_price = fuel_info.iloc[0]['Diesel Price (₹/Liter)']
    else:
        return "Invalid fuel type. Please choose 'petrol' or 'diesel'."
    
    # Step 3: Retrieve Electricity Cost
    electricity_info = pd.DataFrame(data1)
    electricity_cost = electricity_info[electricity_info['state'] == state]['cost'].values
    if len(electricity_cost) == 0:
        return "State not found in electricity data."
    electricity_cost = electricity_cost[0]
    
    # Step 4: Calculate Manual Cost
    manual_cost = fuel_price / mileage * kms_month
    
    # Step 5: Calculate EV Cost
    energy = battery_capacity / range_km
    ev_cost = energy * kms_month * electricity_cost
    
    # Step 6: Calculate Savings
    savings = manual_cost - ev_cost
    
    return {
        'Manual Cost (₹)': round(manual_cost, 2),
        'EV Cost (₹)': round(ev_cost, 2),
        'Savings (₹)': round(savings, 2)
    }

# User input prompts
car_name = input("Enter the Car Name: ")
fuel_type = input("Enter the fuel type (petrol/diesel): ")
state = input("Enter the state: ")
kms_month = int(input("Enter the kilometers driven per month: "))
mileage = float(input("Enter the mileage of the car (km per liter): "))

# Call the savings calculator with user inputs
result = savings_calculator(car_name, fuel_type, state, kms_month, mileage)
print(result)


Enter the Car Name: BYD Seal Premium Range
Enter the fuel type (petrol/diesel): petrol
Enter the state: Karnataka
Enter the kilometers driven per month: 2000
Enter the mileage of the car (km per liter): 12
{'Manual Cost (₹)': 17240.0, 'EV Cost (₹)': 2083.05, 'Savings (₹)': 15156.95}
